I start off by importing the libraries I'm going to need for webscraping, EDA, and modeling. I also import my custom functions from a .py file

In [1]:
from selenium.webdriver import Chrome
import json
import pandas as pd
import gensim
import pickle
import functions as fc

I then set up the web scraper with ease of access to a web blocker to install on the selenium instance. I do this because ads interfered with my webscraper on early attemtps. the urls below link to the main catagories on Allrecipes.com

In [2]:
# browser = Chrome()


# urls = ["https://www.allrecipes.com/recipes/76/appetizers-and-snacks/",
#         "https://www.allrecipes.com/recipes/156/bread/",
#         "https://www.allrecipes.com/recipes/78/breakfast-and-brunch/",
#         "https://www.allrecipes.com/recipes/276/desserts/cakes/",
#         "https://www.allrecipes.com/recipes/201/meat-and-poultry/chicken/",
#         "https://www.allrecipes.com/recipes/362/desserts/cookies/",
#         "https://www.allrecipes.com/recipes/430/seafood/shellfish/shrimp/",
#         "https://www.allrecipes.com/recipes/253/everyday-cooking/slow-cooker/",
#         "https://www.allrecipes.com/recipes/1227/everyday-cooking/vegan/",
#         "https://www.allrecipes.com/recipes/86/world-cuisine/"]

# browser.set_page_load_timeout(300)
# browser.get('https://chrome.google.com/webstore/detail/adblock/gighmmpiobklfepjocnamgkkbiglidom')

Below I have the selenium instance go to each url. Each url is a list page that doesn't load until you scrol down, so I have the instance scroll down a handful of times, then copy the links from each list entry. Once this is done, the instance goes from link to link to copy the recipe information from the appropriate html tags


In [3]:
# for url in urls:
#     print(url)
#     browser.get(url)

#     for _ in range(6):
#         browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(3 + random.random()*10)

#     link_list = browser.find_elements_by_xpath('//*[contains(@class,"fixed-recipe-card__info")]/a')

#     recipe_page_list = [link.get_attribute('href') for link in link_list]

#     len(recipe_page_list)

#     with open('recipes_list.json', 'a') as f:


#         for recipe in recipe_page_list:

#             browser.get(recipe)

#             recipe_dict = {}
            
#             try:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[@id="recipe-main-content"]').text
#             except:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[contains(@class, "intro article-info")]/h1').text
            
#             try:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@id,"lst_ingred")]')
#                 item_list = []
#                 for element in lala_list:

#                     item_list += element.find_elements_by_css_selector('span')

#                 ingredients = [item.text for item in item_list[:-1]]
#                 recipe_dict['ingredients'] = ingredients
            
#             except:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@class, "ingredients-item-name")]')

#                 ingredients = [item.text for item in lala_list]
#                 recipe_dict['ingredients'] = ingredients

#             json.dump(recipe_dict, f)
#             f.write('\n')
#             time.sleep(3 + random.random()*10)

Now I bring the data in line by line from the json file it was stored in during scraping

In [4]:
data = []
with open('recipes_list.json') as file:
    for line in file:
        data.append(json.loads(line))


Load it into a pandas dataframe for EDA

In [5]:
df = pd.DataFrame(data)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 2 columns):
title          929 non-null object
ingredients    929 non-null object
dtypes: object(2)
memory usage: 14.6+ KB


Some of the recipes scraped came back empty. Below I find their indices

In [7]:
indices_list = []
for i in range(len(df.ingredients)):
    
        try: 
            if df.ingredients[i][0] == '':
                print(df.ingredients[i])
                indices_list.append(i)
                
        except IndexError:
            print(df.ingredients[i])
            indices_list.append(i)


print(indices_list)

[]
['', '', '', '', '', '', '', '', '', '']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[60, 136, 217, 220, 222, 251, 436, 441, 509, 710, 713, 731, 778, 788, 867]


Drop the empty recipes and reindex the dataframe

In [8]:
to_be_reindexed_df = df.drop(indices_list)

clean_df = to_be_reindexed_df.reset_index(drop=True)
clean_df.head()

,title,ingredients
0,Grilled Shrimp Scampi Cocktail,"[2 pounds unpeeled large shrimp, 2 tablespoons..."
1,Easy Rumaki with Pineapple,"[cooking spray, 24 (1 inch) cubes fresh pineap..."
2,Perfect Crab-Stuffed Mushrooms,"[2 tablespoons butter, 2 tablespoons minced gr..."
3,Baked Buffalo Chicken Dip,"[3 cups diced cooked rotisserie chicken, 2 (8 ..."
4,Strawberry Bruschetta,"[24 slices French baguette, 1 tablespoon butte..."


In [9]:
clean_df.ingredients[0]

['2 pounds unpeeled large shrimp',
 '2 tablespoons olive oil',
 '1 teaspoon minced garlic',
 '1/4 teaspoon salt',
 '1/8 teaspoon freshly ground black pepper',
 '1 cup cocktail sauce',
 '1 lemon, cut into wedges']

Run my cleaning functions on the ingredients, removing amounts and measurements, as well as cooking stop words and some brand names. Then split into individual words

In [10]:
clean_df.ingredients = clean_df.ingredients.map(fc.clean_ingredients)

def lower_list_strings(list_object):
        
    low_words = []
    for word in list_object:
        low_words.append(word.lower())
    return low_words

clean_df.ingredients = clean_df.ingredients.map(lower_list_strings)

Vectorize ingredient words

In [11]:
model = gensim.models.Word2Vec(clean_df.ingredients, min_count=1, sg=1, iter=5)

model.train(clean_df.ingredients,
            total_examples=model.corpus_count,
            epochs=model.epochs)

(36090, 68530)

In [12]:
model.corpus_total_words

13706

In [13]:
model.wv

Test to see similar ingredient vectors

In [14]:
model.wv.most_similar(positive=["chicken", "cheese"])

[('mozzarella', 0.9946740865707397),
 ('swiss', 0.9834051132202148),
 ('parmesan', 0.9799907207489014),
 ('mayonnaise', 0.9752441048622131),
 ('ham', 0.9751714468002319),
 ('mushrooms', 0.9749267101287842),
 ('artichoke', 0.9745723605155945),
 ('spinach', 0.9745497703552246),
 ('cheddar', 0.9702599048614502),
 ('jack', 0.9699516296386719)]

Function to return recipes with ingredient vectors similar to input vectors

In [15]:
fc.sound_tasty(model, clean_df.ingredients, clean_df.title)

chicken
cheese
turkey
bacon


[{'Title': 'Hot Artichoke and Spinach Dip II',
  'Ingreds': ['cream',
   'cheese',
   'mayonnaise',
   'parmesan',
   'cheese',
   'romano',
   'cheese',
   'clove',
   'garlic',
   'basil',
   'garlic',
   'salt',
   'salt',
   'pepper',
   'artichoke',
   'spinach',
   'mozzarella',
   'cheese']},
 {'Title': 'Amazing No Cook Spinach Artichoke Dip',
  'Ingreds': ['onion',
   'garlic',
   'artichoke',
   'spinach',
   'parmesan',
   'cheese',
   'vegetable',
   'soup',
   'mix',
   'cream',
   'cheese',
   'mayonnaise',
   'sour',
   'cream']},
 {'Title': 'Florentine Artichoke Dip',
  'Ingreds': ['spinach',
   'artichoke',
   'garlic',
   'mayonnaise',
   'cream',
   'cheese',
   'lemon',
   'juice',
   'parmesan',
   'cheese']},
 {'Title': 'Luscious Spinach Artichoke Dip',
  'Ingreds': ['artichoke',
   'spinach',
   'sour',
   'cream',
   'mayonnaise',
   'cream',
   'cheese',
   'romano',
   'cheese',
   'garlic']},
 {'Title': 'Antipasto Squares',
  'Ingreds': ['dinner',
   'rolls',


Save model and columns to pickle files

In [17]:
pickle.dump(model, open('ingred_model.pkl', 'wb'))

pickle.dump(clean_df.ingredients, open('ingredients.pkl', 'wb'))

pickle.dump(clean_df.title, open('title.pkl', 'wb'))